In [1]:
incremental_flag = spark.conf.get("incremental_flag", "0")

print("incremental_flag =", incremental_flag)

if incremental_flag == "1":
    print("Incremental load logic")
else:
    print("Full load logic")

StatementMeta(, e12b91ad-e3ad-4e19-b844-bdc76bf822a1, 3, Finished, Available, Finished)

incremental_flag = 0
Full load logic


# Create Dimensional Model

In [3]:
df_src = spark.sql('''
SELECT DISTINCT(Model_ID)as Model_ID,Product_Name 
FROM parquet.`abfss://Azure_project@onelake.dfs.fabric.microsoft.com/SilverLH.Lakehouse/Files/carsales`
''')

StatementMeta(, 7b6b65e3-4657-4855-af1e-f74e4efeeb95, 5, Finished, Available, Finished)

In [7]:
display(df_src)

StatementMeta(, da5e2ab9-fbfb-42f7-8c5d-08ae4e07b0fa, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b77d6958-98bf-45d5-8e7b-9b9e850d45a1)

In [5]:
if spark.catalog.tableExists("dbo.sales"):

    df_sink = spark.sql("""
        SELECT 
            Model_ID,
            Product_Name
        FROM parquet.`abfss://Azure_project@onelake.dfs.fabric.microsoft.com/SilverLH.Lakehouse/Files/carsales`
    """)

else:

    df_sink = spark.sql("""
        SELECT 
            CAST(NULL AS BIGINT) AS dim_model_key,
            Model_ID,
            Product_Name
        FROM parquet.`abfss://Azure_project@onelake.dfs.fabric.microsoft.com/SilverLH.Lakehouse/Files/carsales`
        WHERE 1 = 0
    """)

StatementMeta(, 7b6b65e3-4657-4855-af1e-f74e4efeeb95, 7, Finished, Available, Finished)

In [6]:
display(df_sink)

StatementMeta(, 7b6b65e3-4657-4855-af1e-f74e4efeeb95, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1fe09389-4ccf-477c-9caf-34ecb870233c)

In [2]:
incremental_flag = spark.conf.get("incremental_flag", "0")
print("incremental_flag =", incremental_flag)

StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 4, Finished, Available, Finished)

incremental_flag = 0


In [3]:
if incremental_flag == "0":
    max_value = 0
else:
    max_value_df = spark.sql("""
        SELECT MAX(dim_model_key) AS max_key
        FROM dbo.sales
    """)
    max_value = max_value_df.collect()[0][0]

print("max_value =", max_value)

StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 5, Finished, Available, Finished)

max_value = 0


In [4]:
if incremental_flag == "0":
    max_value = 0
else:
    max_value_df = spark.sql("SELECT COALESCE(MAX(dim_model_key), 0) FROM dbo.sales")
    max_value = max_value_df.collect()[0][0]

StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 6, Finished, Available, Finished)

In [10]:
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 12, Finished, Available, Finished)

In [17]:
df_src = spark.read.parquet(
    "abfss://Azure_project@onelake.dfs.fabric.microsoft.com/SilverLH.Lakehouse/Files/carsales"
)


StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 19, Finished, Available, Finished)

In [18]:
try:
    df_sink = spark.sql("SELECT Model_ID, dim_model_key FROM dbo.sales")
except:
    df_sink = spark.createDataFrame(
        [],
        "Model_ID STRING, dim_model_key LONG"
    )

StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 20, Finished, Available, Finished)

In [19]:
from pyspark.sql.functions import col

df_filter = (
    df_src.alias("src")
    .join(
        df_sink.alias("snk"),
        col("src.Model_ID") == col("snk.Model_ID"),
        "left"
    )
    .where(col("snk.Model_ID").isNull())
    .select(
        col("src.Model_ID"),
        col("src.Product_Name")
    )
)

StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 21, Finished, Available, Finished)

In [22]:
df_dim_candidates = df_filter.select(
    "Model_ID",
    "Product_Name"
).distinct()

StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 24, Finished, Available, Finished)

In [23]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

w = Window.orderBy("Model_ID")

df_filter_new = df_dim_candidates.withColumn(
    "dim_model_key",
    row_number().over(w) + max_value
)

StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 25, Finished, Available, Finished)

In [24]:
df_filter_new.show()

StatementMeta(, a2382ba8-a204-4f9e-8a69-461d4d0bb6cf, 26, Finished, Available, Finished)

+--------+------------+-------------+
|Model_ID|Product_Name|dim_model_key|
+--------+------------+-------------+
| Acu-M59|       Acura|            1|
| Acu-M60|       Acura|            2|
| Acu-M61|       Acura|            3|
| Acu-M62|       Acura|            4|
| Acu-M63|       Acura|            5|
|Agr-M111|      Agrale|            6|
|Agr-M112|      Agrale|            7|
|Agr-M113|      Agrale|            8|
|Aud-M227|        Audi|            9|
|Aud-M228|        Audi|           10|
|Aud-M229|        Audi|           11|
|Aud-M230|        Audi|           12|
|Aud-M231|        Audi|           13|
|Aud-M232|        Audi|           14|
|Aud-M233|        Audi|           15|
|Aud-M234|        Audi|           16|
|Aud-M235|        Audi|           17|
|Aud-M236|        Audi|           18|
|Aud-M237|        Audi|           19|
|Aud-M238|        Audi|           20|
+--------+------------+-------------+
only showing top 20 rows



# UPSERT

In [9]:
from delta.tables import DeltaTable
from pyspark.sql.functions import col


StatementMeta(, 7b6b65e3-4657-4855-af1e-f74e4efeeb95, 11, Finished, Available, Finished)

In [7]:
df_src   # source data
df_sink  # existing dimension table (can be empty on initial run)

StatementMeta(, 7b6b65e3-4657-4855-af1e-f74e4efeeb95, 9, Finished, Available, Finished)

DataFrame[Model_ID: string, Product_Name: string]

In [11]:
from pyspark.sql.functions import col

df_filter = (
    df_src.alias("src")
    .join(
        df_sink.alias("snk"),
        col("src.Model_ID") == col("snk.Model_ID"),
        "left"
    )
    .select(
        col("src.Model_ID"),
        col("src.Product_Name")
    )
)

StatementMeta(, 7b6b65e3-4657-4855-af1e-f74e4efeeb95, 13, Finished, Available, Finished)

In [14]:
df_filter_new.show()                                                                                                                                                                                                                                                                                                                                                                

StatementMeta(, 7b6b65e3-4657-4855-af1e-f74e4efeeb95, 16, Finished, Available, Finished)

+--------+------------+-------------+
|Model_ID|Product_Name|dim_model_key|
+--------+------------+-------------+
| Acu-M59|       Acura|            1|
| Acu-M60|       Acura|            2|
| Acu-M61|       Acura|            3|
| Acu-M62|       Acura|            4|
| Acu-M63|       Acura|            5|
|Agr-M111|      Agrale|            6|
|Agr-M112|      Agrale|            7|
|Agr-M113|      Agrale|            8|
|Aud-M227|        Audi|            9|
|Aud-M228|        Audi|           10|
|Aud-M229|        Audi|           11|
|Aud-M230|        Audi|           12|
|Aud-M231|        Audi|           13|
|Aud-M232|        Audi|           14|
|Aud-M233|        Audi|           15|
|Aud-M234|        Audi|           16|
|Aud-M235|        Audi|           17|
|Aud-M236|        Audi|           18|
|Aud-M237|        Audi|           19|
|Aud-M238|        Audi|           20|
+--------+------------+-------------+
only showing top 20 rows

